In [1]:
# !pip install albumentations

In [2]:
import urllib
import tarfile
import os
import wget
import glob

import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import tarfile

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pytorch_ssim as ssi
l2 = nn.MSELoss()
l1= nn.L1Loss()
NUM_EPOCHS = 200

from Models import model1, model2, model3, model4, model5, model6

#augmentation library
import albumentations as A

%matplotlib inline

In [3]:
url = 'http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar'
file_tmp = wget.download(url)
tar = tarfile.open(file_tmp)
tar.extractall()
photos= glob.glob('VOCdevkit/VOC2007/JPEGImages/*.jpg')

In [4]:
sample = []
[sample.append(cv2.imread(photos[ind])) for ind in range(len(photos))]
sample = np.array(sample)
y_large = [cv2.resize(sample[ind] , (288,288)) for ind in range(len(sample))]
y_mid = [cv2.resize(y_large[ind] , (144,144)) for ind in range(len(sample))]
x_train = [cv2.resize(y_large[ind] , (72,72)) for ind in range(len(sample))]

In [5]:
inds = np.random.choice(len(photos),len(photos),False)
first_80_percent = int(.8*len(photos))

X_train, y_mid_train, y_large_train = np.array(x_train)[inds[:first_80_percent]], np.array(y_mid)[inds[:first_80_percent]], np.array(y_large)[inds[:first_80_percent]]

X_test, y_mid_test, y_large_test = np.array(x_train)[inds[first_80_percent:]], np.array(y_mid)[inds[first_80_percent:]], np.array(y_large)[inds[first_80_percent:]]

In [6]:
 class DS_mid(Dataset):
    def __init__(self,df,labels, transforms=None):
        super().__init__()
        self.df=df
        self.labels=labels
        self.transforms=transforms
 
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        img= self.df[idx]
        label = self.labels[idx]        
        if not self.transforms is None:
            transformed_img = self.transforms(image=img)
            img = transformed_img['image']
        return img,label
    
class DS_both(Dataset):
    def __init__(self,df,labels_mid,labels_large, transforms=None):
        super().__init__()
        self.df=df
        self.labels_mid=labels_mid
        self.labels_large=labels_large
        self.transforms=transforms

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        img= self.df[idx]
        labels_mid = self.labels_mid[idx]   
        labels_large = self.labels_large[idx]        
        if not self.transforms is None:
            transformed_img = self.transforms(image=img)
            img = transformed_img['image']
        return img,labels_mid,labels_large


In [7]:
def train_fold_mid(tr_loader,model,criterion,optimizer, num_epochs):
    model.to(device)
    
    training_log =[]

    
    for epoch in range(num_epochs):
        print("started training epoch no. {}".format(epoch+1))
        tr_loss = 0
        for step,batch in enumerate(tr_loader):
            imgs, labels = batch
            imgs=imgs.to(device,dtype=torch.float32)
            labels = labels.to(device,dtype=torch.float32)
            outputs = model(imgs)

            loss = 1-criterion(outputs, labels, window_size=12)
            loss.backward()
            tr_loss+=loss.item()
            optimizer.step()
            optimizer.zero_grad()

        training_log.append({
                'epoch':epoch,
                'train_loss':tr_loss / len(tr_loader),
                })

    return training_log

def test_evaluation_mid(tst_loader,model,losses):  
    model.eval()

    loss_ssim = 0
    loss_mse = 0
    loss_mae = 0
    N=0

    for step,batch in enumerate(tst_loader):
        imgs, labels = batch
        
        
        imgs = imgs.to(device,dtype=torch.float32)
        labels = labels.to(device,dtype=torch.float32)
        outputs = model(imgs)

        loss_ssim += 1-losses[0](outputs,labels, window_size=12).item()
        loss_mse += losses[1](outputs,labels).item()
        loss_mae += losses[2](outputs,labels).item()
        N += len(batch)
    f,ax = plt.subplots(batch[0].shape[0], 3 ,figsize = (5*batch[0].shape[0],15) )
    
    imgs , labels= imgs.cpu().detach().numpy().astype(int), labels.cpu().detach().numpy().astype(int)
    outputs_hats = np.abs(outputs.cpu().detach().numpy().astype(int))
    
#     for i in range(batch[0].shape[0]):

#         orig_x , orig_y= imgs[i], labels[i]
#         outputs = outputs_hats[i]
#         val=0
#         ax[i,val].imshow(orig_x)
#         ax[i,val].set_title('original 72*72 image')

#         val+=1
#         ax[i,val].imshow(outputs)
#         ax[i,val].set_title('model output')

#         val+=1
#         ax[i,val].imshow(orig_y)
#         ax[i,val].set_title('target image')
    test_loss = {'ssim':loss_ssim/N,"mse": loss_mse/N,"mae": loss_mae/N}
    
    return imgs, labels, outputs_hats, test_loss 
#   print ("test set results:\n SSIM:{}\tL1 loss(MSE):{}\tL2 loss(MAE):{}".format(loss_ssim/N, loss_mse/N, loss_mae/N))

In [10]:
def train_fold_both(tr_loader,model,criterion,optimizer, num_epochs):
    model.to(device)
    
    training_log =[]
    
    for epoch in range(num_epochs):
        print("started training epoch no. {}".format(epoch+1))
        tr_loss = 0
        for step,batch in enumerate(tr_loader):
            imgs, labels_mid, labels_large = batch
            imgs=imgs.to(device,dtype=torch.float32)
            labels_mid,labels_large = labels_mid.to(device,dtype=torch.float32), labels_large.to(device,dtype=torch.float32)
            outputs_1, outputs_2 = model(imgs)

            loss = (1-criterion(outputs_1, labels_mid, window_size=12)) + (1-criterion(outputs_2, labels_large, window_size=12))
            loss.backward()
            tr_loss+=loss.item()
            optimizer.step()
            optimizer.zero_grad()

        training_log.append({
                'epoch':epoch,
                'train_loss':tr_loss / len(tr_loader),
                })

    return training_log


def test_evaluation_both(tst_loader,model,losses):  
    model.eval()

    loss_ssim = 0
    loss_mse = 0
    loss_mae = 0
    N=0

    for step,batch in enumerate(tst_loader):
        imgs, labels_mid, labels_large = batch
        
        
        imgs = imgs.to(device,dtype=torch.float32)
        labels_mid, labels_large = labels_mid.to(device,dtype=torch.float32), labels_large.to(device,dtype=torch.float32)
        outputs = model(imgs)

        loss_ssim += (1-losses[0](outputs[0],labels_mid, window_size=12).item()) + (1-losses[0](outputs[1],labels_large, window_size=12).item())
        loss_mse += losses[1](outputs[0],labels_mid).item() + (losses[1](outputs[1],labels_large).item())
        loss_mae += losses[2](outputs[0],labels_mid).item() + losses[2](outputs[1],labels_large).item()
        N += len(batch)
    f,ax = plt.subplots(batch[0].shape[0], 5 ,figsize = (5*batch[0].shape[0],25) )
    
    imgs , labels_mid , labels_large = imgs.cpu().detach().numpy().astype(int), labels_mid.cpu().detach().numpy().astype(int), labels_large.cpu().detach().numpy().astype(int)
    outputs_small,outputs_large = np.abs(outputs[0].cpu().detach().numpy().astype(int)),np.abs(outputs[1].cpu().detach().numpy().astype(int))
    test_loss = {'ssim':loss_ssim/N,"mse": loss_mse/N,"mae": loss_mae/N}

#     for i in range(batch[0].shape[0]):
#         orig_x , orig_mid, orig_large= imgs[i], labels_mid[i], labels_large[i]
#         output1 = outputs_small[i]
#         output2 = outputs_large[i]
#         val=0
#         ax[i,val].imshow(orig_x)
#         ax[i,val].set_title('original 72*72 image')

#         val+=1
#         ax[i,val].imshow(output1)
#         ax[i,val].set_title('model output small')

#         val+=1
#         ax[i,val].imshow(orig_mid)
#         ax[i,val].set_title('target image(mid)')

#         val+=1
#         ax[i,val].imshow(output2)
#         ax[i,val].set_title('model output large')


#         val+=1
#         ax[i,val].imshow(orig_large)
#         ax[i,val].set_title('target image(large)')
    
    return imgs , labels_mid , labels_large, outputs_small, outputs_large, test_loss
#     print ("test set results:\n SSIM:{}\tL1 loss(MSE):{}\tL2 loss(MAE):{}".format(loss_ssim/N, loss_mse/N, loss_mae/N))
        

## MODEL 1

In [11]:
train_set = DS_mid(X_train,y_mid_train)
test_set= DS_mid(X_test,y_mid_test)

train_loader=DataLoader(dataset= train_set, batch_size = 4, shuffle = True, num_workers = 2)
test_loader=DataLoader(dataset= test_set, batch_size = 4, shuffle = True, num_workers = 2)

In [12]:
model = model1.firstmodel()
criterion = ssi.ssim
optimizer = optim.Adam(model.parameters(), lr=0.001)
if torch.has_cuda:
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu:0')

In [ ]:
m1_train_log = train_fold_mid(train_loader,model,criterion,optimizer,num_epochs= NUM_EPOCHS)
m1_imgs, m1_labels, m1_output_hats, m1_test_loss = test_evaluation_mid(test_loader,model,[criterion,l2,l1])

started training epoch no. 1


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify

started training epoch no. 2


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify

started training epoch no. 3


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify

started training epoch no. 4


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify

started training epoch no. 5


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify

## MODEL 2

In [ ]:
train_set = DS_both(X_train,y_mid_train,y_large_train)
test_set= DS_both(X_test,y_mid_test, y_large_test)

train_loader=DataLoader(dataset= train_set, batch_size = 4, shuffle = True, num_workers = 2)
test_loader=DataLoader(dataset= test_set, batch_size = 4, shuffle = True, num_workers = 2)

In [ ]:
model= model2.second_model()

criterion = ssi.ssim
optimizer = optim.Adam(model.parameters(), lr=0.001)

if torch.has_cuda:
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu:0')

In [ ]:
m2_train_log = train_fold_both(train_loader,model,criterion,optimizer,num_epochs= NUM_EPOCHS)
m2_imgs , m2_labels_mid , m2_labels_large, m2_outputs_small, m2_outputs_large, m2_test_loss = test_evaluation_both(test_loader,model,[criterion,l2,l1])

## Model 3

In [ ]:
model= model3.third_model()
criterion = ssi.ssim
optimizer = optim.Adam(model.parameters(), lr=0.001)

if torch.has_cuda:
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu:0')

In [ ]:
m3_train_log = train_fold_both(train_loader,model,criterion,optimizer,num_epochs= NUM_EPOCHS)
m3_imgs , m3_labels_mid , m3_labels_large, m3_outputs_small, m3_outputs_large, m3_test_loss = test_evaluation_both(test_loader,model,[criterion,l2,l1])

## Model 4

In [ ]:
model= model4.fourth_model()
criterion = ssi.ssim
optimizer = optim.Adam(model.parameters(), lr=0.001)

if torch.has_cuda:
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu:0')

In [ ]:
m4_train_log = train_fold_both(train_loader,model,criterion,optimizer,num_epochs= NUM_EPOCHS)
m4_imgs , m4_labels_mid , m4_labels_large, m4_outputs_small, m4_outputs_large, m4_test_loss = test_evaluation_both(test_loader,model,[criterion,l2,l1])

## Model 5

In [ ]:
model= model5.fifth_model()
criterion = ssi.ssim
optimizer = optim.Adam(model.parameters(), lr=0.001)

if torch.has_cuda:
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu:0')

In [ ]:
m5_train_log = train_fold_both(train_loader,model,criterion,optimizer,num_epochs= NUM_EPOCHS)
m5_imgs , m5_labels_mid , m5_labels_large, m5_outputs_small, m5_outputs_large, m5_test_loss = test_evaluation_both(test_loader,model,[criterion,l2,l1])

## BONUS - Model 6

In [ ]:
model= model6.sixth_model()
criterion = ssi.ssim
optimizer = optim.Adam(model.parameters(), lr=0.001)

if torch.has_cuda:
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu:0')

In [ ]:
m6_train_log = train_fold_both(train_loader,model,criterion,optimizer,num_epochs= NUM_EPOCHS)
m6_imgs , m6_labels_mid , m6_labels_large, m6_outputs_small, m6_outputs_large, m6_test_loss = test_evaluation_both(test_loader,model,[criterion,l2,l1])

In [ ]:
def plot_both(model_name,imgs,labels_mid,outputs_mid,labels_large=None,outputs_large=None):
    for i in range(imgs.shape[0]):
        
        if labels_large:
            f,ax = plt.subplots(imgs.shape[0], 5 ,figsize = (5*imgs.shape[0],25) )
        else:
            f,ax = plt.subplots(imgs.shape[0], 3 ,figsize = (5*imgs.shape[0],15) )
        
        plt.title(model_name)
        
        orig_x = imgs[i]
        orig_mid = labels_mid[i]
        if labels_large:
            orig_large = labels_large[i]
        output1 = outputs_mid[i]
        if outputs_large:
            output2 = outputs_large[i]
            
        val=0
        ax[i,val].imshow(orig_x)
        ax[i,val].set_title('original 72*72 image')

        val+=1
        ax[i,val].imshow(output1)
        ax[i,val].set_title('model output small')

        val+=1
        ax[i,val].imshow(orig_mid)
        ax[i,val].set_title('target image(mid)')

        if labels_large:
            val+=1
            ax[i,val].imshow(output2)
            ax[i,val].set_title('model output large')
            
        if outputs_large:
            val+=1
            ax[i,val].imshow(orig_large)
            ax[i,val].set_title('target image(large)')
            
        plt.show()

## REPORT

In [ ]:
# Graphing the train loss

fig, ax = plt.subplots(figsize=(12,8))

for i in range(1,7):
    train_log = eval("m{}_train_log".format(i))
    train_log = pd.DataFrame(train_log)['train_loss']
    sns.lineplot(train_log.index, train_log, label="model {}".format(i))

In [ ]:
# Dataframe of PSNR metrics on test loss

for i in range(1,7):
    eval("df{}".format(i)) = pd.DataFrame(eval("m{}_test_loss".format(i)))
df = pd.concat((df1,df2,df3,df4,df5,df6),axis=1)
df.index = [1,2,3,4,5,6]


In [ ]:
# Show examples of each models output

for i in range(1,7):
    if i == 1:
        m_imgs, m_labels_mid, m_outputs_small = m1_imgs, m1_labels, m1_output_hats
        m_labels_large = m_outputs_large = None
    else:
        m_imgs , m_labels_mid , m_labels_large, m_outputs_small, m_outputs_large = eval("m{}_imgs".format(i)) , eval("m{}_labels_mid".format(i)) , eval("m{}_labels_large".format(i)), eval("m{}_outputs_small".format(i), eval("m{}_outputs_large".format(i))
    model_name = "Model {}".format(i)
    plot_both(model_name,m_imgs,m_labels_mid,m_outputs_small,m_labels_large,m_outputs_large)